Notebook to send jobs to the Ubelix HPC cluster at the University of Bern

In [1]:
import sys
import os
import logging
import glob
import pickle
import json
from pathlib import Path
import rpy2.robjects as ro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

%load_ext rpy2.ipython

logging.basicConfig(filename='example.log', 
                    encoding='utf-8', level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

#datapath = Path("../data")
datapath = Path("/storage/homefs/pd21v747/datanew")

modpath = Path("../scripts")
sys.path.append(os.path.relpath(modpath))

#from misc import Timer, pickler, open_table

In [2]:
sites = {"thyroid": "THCA",
         "lung": "LUAD",
         "kidney": "KIRC",
         "colorectal": "COAD",
         "breast": "BRCA",
         "liver": "LIHC",
         "prostate": "PRAD"}

datasets = {sites[s]: {} for s in sites}

for s in sites:
    f = Path(f"{datapath}/{s}/{sites[s]}/{sites[s]}.csv")
    df = pd.read_csv(f, index_col=0)
    datasets[sites[s]]["genes"] = len(df)
    datasets[sites[s]]["site"] = s
    datasets[sites[s]]["datapath"] = f
    datasets[sites[s]]["outpath"] = f.parent
    datasets[sites[s]]["patients"] = len(df.columns)//2
    print(f"{s:<10}", datasets[sites[s]]["genes"], datasets[sites[s]]["patients"])
    
# Pretty names
cleanout = {"jk": "ReBoost",
            "pcah": "rPCA",
            "none": "None"}
cleandea = {"edger": "edgeR QLF",
            "edgerlrt": "edgeR LRT",
            "deseq2": "DESeq2"}

thyroid    18953 58
lung       18235 52
kidney     18489 72
colorectal 17336 39
breast     18758 111
liver      15611 50
prostate   18908 51


# Define ground truth

Define ground truth DEGs for a given FDR, logFC cutoff as the intersection of DEGs from all three DEA tests (Wald, LRT, QLF)

In [7]:
import sys, importlib
importlib.reload(sys.modules["DEA"])

from DEA import run_dea_on_full_data
from process import find_ground_truth

DEAs = ["edgerlrt","edgerqlf", "deseq2"]
FDRs = [0.1,0.05,0.01,0.001]
logFCs = [0, 1] # formal lfc threhsold in edger or deseq2
logFCs_post = [0,0.5,1,1.5,2] # post hoc threshold

run_dea_on_full_data(datasets, DEAs, overwrite = False, lfcs = logFCs)

overwrite_ = True
datasets = find_ground_truth(datasets, DEAs, FDRs, logFCs_post, lfc_test = 0, overwrite = overwrite_)
datasets = find_ground_truth(datasets, DEAs, FDRs, [1], lfc_test = 1, overwrite = overwrite_)

assert overwrite_ #fix later

Running DEA for THCA edgerqlf
R[write to console]: Error in file(filename, "r", encoding = encoding) : 
  cannot open the connection

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In file(filename, "r", encoding = encoding) :
R[write to console]: 
 
R[write to console]:  cannot open file '../scripts/R_functions.R': No such file or directory

Elapsed time: 0.0112 seconds


RRuntimeError: Error in file(filename, "r", encoding = encoding) : 
  cannot open the connection
